In [1]:
import java.util.Date
import org.apache.spark.sql.SparkSession
import java.io._
import org.apache.spark.{SparkConf, SparkContext}
import util.control.Breaks._
import org.apache.spark.broadcast.Broadcast

In [2]:
    val conf = new SparkConf().setAppName("response_count").setMaster("local[*]")
    @transient
    val sc = new SparkContext(conf)
    val spark = SparkSession.builder.getOrCreate

conf = org.apache.spark.SparkConf@326f4121
sc = org.apache.spark.SparkContext@32ba8b84
spark = org.apache.spark.sql.SparkSession@271c1a44


Spark local-1542309105888: Some(http://chenxijiadembp:4040)

In [3]:
    var raw = sc.textFile("Data/yelp_reviews_clustering_small.txt")
    var feature = "W"
    var N = 5
    var iterations = 20

raw = Data/yelp_reviews_clustering_small.txt MapPartitionsRDD[1] at textFile at <console>:38
feature = W
N = 5
iterations = 20


20

In [19]:
    def word2TF(review:Array[String], word_map:Map[String, Int], Size:Int): Array[Double] ={
      var count: Array[Double] = new Array[Double](Size)
      for(word <- review){
        var word_index = word_map(word)
        count(word_index)+=1.0
      }
      var mostfreq = count.max
      var res = count.map(x=> x / mostfreq)
      res
    }


word2TF: (review: Array[String], word_map: Map[String,Int], Size: Int)Array[Double]


In [20]:
    def word2IDF(unique_word:(String,Int), doc_word_num:List[Array[Int]], Size:Int):(Any,Double) ={
      var app_count = 0
      for(i <- 0 until doc_word_num.length){
        var _doc = doc_word_num(i).toSet
        if (_doc.contains(unique_word._2)){app_count+=1}
      }
      (unique_word._2, Math.log(Size.toDouble/app_count.toDouble))
    }

word2IDF: (unique_word: (String, Int), doc_word_num: List[Array[Int]], Size: Int)(Any, Double)


In [21]:
    var word = raw.map(row => row.split(" "))
    var word_list = word.collect().toList
    var doc_num = word_list.size
    var ret = Set.empty[String]
    var allword = word.flatMap(x=>x).collect().toSet
    var allword_size = allword.size
    var word_index = allword.zipWithIndex.map{case(v,i) => (v, i)}
    var word_map = word_index.toMap
    var doc_word_num = word.map(doc=>{
      var num = doc.map(x=> word_map(x)).toSet.toArray
      num
    }).collect().toList

word = MapPartitionsRDD[23] at map at <console>:46
word_list = List(Array(back, 20052007, place, favorite, thai, place, ever, id, go, alllll, time, never, complaints, started, get, known, got, busy, service, started, suck, portion, sizes, got, cut, half, huge, problem, paying, way, less, food, last, time, went, pork, pad, se, ew, tasted, good, finished, plate, still, hungry, used, know, manager, would, greet, hello, melissa, nice, see, diet, coke, pad, thai, pad, se, ew, days, know, still, knows, disregards, presence, also, asked, new, portion, sizes, answer, great, food, worth, money, havent, back, year, refuse, pay, 1015, dinner, still, hungry, sorry, pinkaow, used), Array(like, thai, food, try, original, thai, bbq, pad, se...


List([Ljava.lang.String;@412be401, [Ljava.lang.String;@2bf93f40, [Ljava.lang.String;@13b078e1, [Ljava.lang.String;@7d4385c0, [Ljava.lang.String;@63d67d85, [Ljava.lang.String;@1374d80, [Ljava.lang.String;@11505a37, [Ljava.lang.String;@70dca058, [Ljava.lang.String;@2036c193, [Ljava.lang.String;@1b56c358, [Ljava.lang.String;@8c4b3e7, [Ljava.lang.String;@1d8f1cc7, [Ljava.lang.String;@ba37cbc, [Ljava.lang.String;@26c23293, [Ljava.lang.String;@b34e627, [Ljava.lang.String;@6e38baa2, [Ljava.lang.String;@7e6e7d45, [Ljava.lang.String;@490f9eb1, [Ljava.lang.String;@65265c69, [Ljava.lang.String;@4ee007c5, [Ljava.lang.String;@43e9cf09, [Ljava.lang.String;@76a62bd3, [Ljava.lang.String;@21b4e47a, [Ljava.lang.String;@6fa411a6, [Ljava.lang.String;@62381adc, [Ljava.lang.String;@25074dfb, [Ljava.lang.String;@2316ea15, [Ljava.lang.String;@6cf0f6f0, [Ljava.lang.String;@4e3cef48, [Ljava.lang.String;@514b1be1, [Ljava.lang.String;@4519159f, [Ljava.lang.String;@2eb52cba, [Ljava.lang.String;@75ad08b, [Ljava.lan

In [22]:
    var IDF = word_index.map(unique_word=>{word2IDF(unique_word, doc_word_num, doc_num)}).toMap//((word_int, idf)...)


IDF = Map(14221 -> 4.8283137373023015, 2163 -> 5.298317366548036, 8607 -> 6.214608098422191, 645 -> 6.214608098422191, 13950 -> 5.298317366548036, 892 -> 6.907755278982137, 69 -> 6.907755278982137, 10822 -> 6.907755278982137, 5385 -> 6.907755278982137, 5810 -> 5.809142990314028, 7375 -> 6.907755278982137, 5659 -> 6.907755278982137, 9929 -> 5.809142990314028, 11866 -> 6.907755278982137, 12928 -> 6.907755278982137, 2199 -> 5.809142990314028, 8062 -> 5.809142990314028, 3021 -> 6.907755278982137, 13650 -> 6.907755278982137, 8536 -> 6.907755278982137, 5437 -> 5.809142990314028, 1322 -> 6.907755278982137, 1665 -> 4.710530701645918, 5509 -> 6.214608098422191, 5686 -> 6.214608098422191, 1036 -> 5.809142990314028, 9982 -> 3.8632328412587142, 2822 -> 6....


Map(14221 -> 4.8283137373023015, 2163 -> 5.298317366548036, 8607 -> 6.214608098422191, 645 -> 6.214608098422191, 13950 -> 5.298317366548036, 892 -> 6.907755278982137, 69 -> 6.907755278982137, 10822 -> 6.907755278982137, 5385 -> 6.907755278982137, 5810 -> 5.809142990314028, 7375 -> 6.907755278982137, 5659 -> 6.907755278982137, 9929 -> 5.809142990314028, 11866 -> 6.907755278982137, 12928 -> 6.907755278982137, 2199 -> 5.809142990314028, 8062 -> 5.809142990314028, 3021 -> 6.907755278982137, 13650 -> 6.907755278982137, 8536 -> 6.907755278982137, 5437 -> 5.809142990314028, 1322 -> 6.907755278982137, 1665 -> 4.710530701645918, 5509 -> 6.214608098422191, 5686 -> 6.214608098422191, 1036 -> 5.809142990314028, 9982 -> 3.8632328412587142, 2822 -> 6.214608098422191, 7304 -> 6.907755278982137, 14852 -> 3.7722610630529876, 10007 -> 6.907755278982137, 9131 -> 6.907755278982137, 2630 -> 6.907755278982137, 13052 -> 3.611918412977808, 6085 -> 6.907755278982137, 13266 -> 4.605170185988092, 3873 -> 6.90775

In [12]:
var m = array.max

m = 2.0


2.0

In [13]:
    var array = Array[String]("aa","bb","aa")
    var immutableMap = Map("aa" -> 0, "bb" -> 1)
    var Size = 2

array = Array(aa, bb, aa)
immutableMap = Map(aa -> 0, bb -> 1)
Size = 2


2

In [14]:
var res = word2TF(array, immutableMap, Size)

res = Array(2.0, 1.0)


[2.0, 1.0]